In [1]:
import pandas as pd
import json
import ast
from datetime import datetime
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import random

In [ ]:
# Leer el contenido del archivo: output_steam_games.json
with open(r'\\STEAM_recommendation\\data\\output_steam_games.json', 'r') as f:
    steam_games_data = [json.loads(line) for line in f]

# Crear el DataFrame
df_steam_games = pd.DataFrame(steam_games_data)

In [2]:
# Leer el contenido del archivo: australian_user_reviews.json
user_reviews = []
with open(r'C:\\Users\\Joaquin\\Desktop\\Henry\\STEAM_recommendation\\data\\australian_user_reviews.json', 'r', encoding='MacRoman') as f:
    for line in f.readlines():
        user_reviews.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_reviews_anidado = pd.DataFrame(user_reviews)

#Desanidar la columna anidada "reviews"
reviews_desanidado = []

for index, row in df_reviews_anidado.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews = row['reviews']

    for i in reviews:
        new_row = {
        'user_id' : user_id,
        'user_url' : user_url,
        'funny' : i.get('funny',''),
        'posted' : i.get('posted',''),
        'last_edited' : i.get('last_edited',''),
        'item_id' : i.get('item_id',''),
        'helpful' : i.get('helpful',''),
        'recommend' : i.get('recommend'),
        'review' : i.get('review')
        }

        reviews_desanidado.append(new_row)

df_user_reviews = pd.DataFrame(reviews_desanidado)

In [ ]:
# Leer el contenido del archivo: australian_users_items.json
with open('/content/drive/MyDrive/Colab Notebooks/PI_data_cleaning/australian_users_items.json', 'r', encoding='utf-8') as f:
    data = f.readlines()

# Convertir las líneas a registros JSON
records = [eval(line.strip()) for line in data]

# Crear el DataFrame a partir de los registros
df_items_anidado= pd.DataFrame(records)

#Desanidar la columna anidada "items"
data_desanidada = []

for index, row in df_items_anidado.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
    items = row['items']

    for i in items:
        new_row = {
        'user_id': user_id,
        'items_count': items_count,
        'steam_id' : steam_id,
        'user_url' : user_url,
        'item_id': i.get('item_id', ''),
        'item_name': i.get('item_name', ''),
        'playtime_forever': i.get('playtime_forever', ''),
        'playtime_2weeks': i.get('playtime_2weeks', '')
        }

        data_desanidada.append(new_row)

df_users_items = pd.DataFrame(data_desanidada)

In [ ]:
# Eliminar las filas con registros NaN en df_steam_games
df_steam_games = df_steam_games.iloc[88310:]

In [ ]:
# Eliminar columnas no útiles en df_steam_games
df_steam_games.drop(columns=['publisher'], inplace=True)
df_steam_games.drop(columns=['title'], inplace=True)
df_steam_games.drop(columns=['url'], inplace=True)
df_steam_games.drop(columns=['tags'], inplace=True)
df_steam_games.drop(columns=['specs'], inplace=True)
df_steam_games.drop(columns=['early_access'], inplace=True)

In [ ]:
# Dividir la lista de la columna tags
df_steam_games = df_steam_games.explode('genres').reset_index(drop=True)
# Convertir la columna tags a strings
df_steam_games['genres'] = df_steam_games['genres'].astype(str)

In [ ]:
# Eliminar las filas que contienen los géneros 'Free to Play' o 'Early Access' o tienen valores NaN en la columna 'genres'
df_steam_games = df_steam_games[~df_steam_games['genres'].str.contains('Free to Play', case=False, na=False)]
df_steam_games = df_steam_games[~df_steam_games['genres'].str.contains('Early Access', case=False, na=False)]
df_steam_games = df_steam_games.dropna(subset=['genres'])

In [ ]:
# Crear un nuevo DataFrame con las columnas 'app_name' y 'genres' sin duplicados
df_genres = df_steam_games[['app_name', 'genres']].drop_duplicates()
# Reiniciar los índices del nuevo DataFrame
df_genres.reset_index(drop=True, inplace=True)

In [ ]:
df_steam_games.drop(columns=['genres'], inplace=True)

In [ ]:
# Convertir la columna 'release_date' al tipo de dato datetime, manejar errores y valores nulos
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')

# Extraer el año y almacenarlo en una nueva columna 'release_year'
df_steam_games['release_year'] = df_steam_games['release_date'].dt.year

# Eliminar la columna 'release_date'
df_steam_games.drop(columns=['release_date'], inplace=True)

# Convertir la columna 'release_year' a tipo de dato integer
df_steam_games['release_year'] = df_steam_games['release_year'].astype('Int64')

In [ ]:
# Reemplazar los valores que contienen 'Free' en la columna 'price' con 0
df_steam_games.loc[df_steam_games['price'].str.contains('Free', case=False, na=False), 'price'] = 0
df_steam_games.loc[df_steam_games['price'].str.contains('Install', case=False, na=False), 'price'] = 0

In [ ]:
# Convertir la columna 'price' en el DataFrame df_steam_games a números (flotantes)
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
# Convertir la columna 'developer' a tipo de dato string (str)
df_steam_games['developer'] = df_steam_games['developer'].astype(str)

In [3]:
# Eliminar columnas no útiles en df_user_reviews
df_user_reviews.drop(columns=['funny'], inplace=True)
df_user_reviews.drop(columns=['last_edited'], inplace=True)
df_user_reviews.drop(columns=['helpful'], inplace=True)

In [4]:
# Elimina la palabra "Posted" y los espacios en blanco al principio y al final
df_user_reviews['posted'] = df_user_reviews['posted'].str.replace('Posted', '').str.strip()

# Define una función para convertir el formato de fecha
def format_posted_date(date_str):
    try:
        # Parsea la fecha en el formato original
        date_obj = datetime.strptime(date_str, '%B %d, %Y.')
        # Formatea la fecha en 'YYYY-MM-DD'
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    except ValueError:
        return None  # En caso de que el formato original no sea válido

# Aplica la función a la columna 'posted' y crea una nueva columna 'formatted_posted'
df_user_reviews['posted_date'] = df_user_reviews['posted'].apply(format_posted_date)
df_user_reviews.drop(columns=['posted'], inplace=True)

In [5]:
# Transformar la columna posted_date a datetime
df_user_reviews['posted_date'] = pd.to_datetime(df_user_reviews['posted_date'], format='%Y-%m-%d', errors='coerce')

# Extraer el año y convertirlo a tipo int
df_user_reviews['posted_year'] = df_user_reviews['posted_date'].dt.year

# Eliminar la columna 'posted_date'
df_user_reviews = df_user_reviews.drop(columns=['posted_date'])

In [6]:
# Eliminar filas con valores nulos en la columna 'posted_year'
df_user_reviews = df_user_reviews.dropna(subset=['posted_year'])

In [7]:
# Eliminar filas con valores nulos en la columna 'posted_year'
df_user_reviews['posted_year'] = df_user_reviews['posted_year'].astype(int)

In [9]:
# Reemplazar True por 1 y False por 0 en la columna 'recommend'
df_user_reviews['recommend'] = df_user_reviews['recommend'].replace({True: 1, False: 0})

In [10]:
# Función para limpiar una cadena de texto
def clean_text(text):
    # Utilizar una expresión regular para eliminar caracteres no alfabéticos o numéricos
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return cleaned_text

# Aplicar la función de limpieza a la columna 'review' del DataFrame
df_user_reviews['review'] = df_user_reviews['review'].apply(clean_text)

In [11]:
# Analisis de sentimiento
# Inicializar el analizador de sentimientos
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(review):
    if isinstance(review, str):
        sentiment = analyzer.polarity_scores(review)
        if sentiment['compound'] >= 0.05:
            return 2  # Positivo
        elif sentiment['compound'] <= -0.05:
            return 0  # Negativo
        else:
            return 1  # Neutral
    else:
        return 1  # Si la reseña no es un string, asumimos que es neutral
    
# Aplicar el análisis de sentimiento y crear la nueva columna 'sentiment_analysis'
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(get_sentiment)

# Reemplazar la columna 'review' con 'sentiment_analysis'
df_user_reviews.drop(columns=['review'], inplace=True)

In [ ]:
# Eliminar columnas con URLs en df_users_items
df_users_items.drop(columns=['user_url'], inplace=True)
df_users_items.drop(columns=['items_count'], inplace=True)
df_users_items.drop(columns=['playtime_2weeks'], inplace=True)
df_users_items.drop(columns=['steam_id'], inplace=True)

In [12]:
# Reemplazar los valores nulos con 0 y luego cambiar el tipo de datos a integer en df_steam_games
df_steam_games['id'] = df_steam_games['id'].fillna(0).astype(int)

# Reemplazar los valores nulos con 0 y luego cambiar el tipo de datos a integer en df_user_reviews
df_user_reviews['item_id'] = df_user_reviews['item_id'].fillna(0).astype(int)

# Reemplazar los valores nulos con 0 y luego cambiar el tipo de datos a integer en df_users_items
df_users_items['item_id'] = df_users_items['item_id'].fillna(0).astype(int)
df_users_items['playtime_forever'] = df_users_items['playtime_forever'].fillna(0).astype(int)

In [ ]:
# Obtener el número de filas actual
n_filas_actual = len(df_users_items)

# Definir el número de filas que deseas mantener
n_filas_mantener = 100000

# Si el número de filas actual es mayor que el objetivo, eliminar filas al azar
if n_filas_actual > n_filas_mantener:
    # Definir el número de filas que deseas eliminar
    n_filas_eliminar = n_filas_actual - n_filas_mantener

    # Seleccionar índices aleatorios de filas a eliminar
    indices_a_eliminar = random.sample(range(n_filas_actual), n_filas_eliminar)

    # Eliminar las filas seleccionadas
    df_users_items = df_users_items.drop(indices_a_eliminar)

In [ ]:
df_steam_games.head()

In [ ]:
df_genres.head()

In [ ]:
df_user_reviews.head()

In [ ]:
df_users_items

In [14]:
# Guarda los DataFrames en archivos JSON
df_steam_games.to_csv('steam_games_clean.csv', index=False)
df_user_reviews.to_csv('user_reviews_clean.csv', index=False)
df_users_items.to_csv('users_items_clean.csv', index=False)
df_genres.to_csv('genres_clean.csv', index=False)